In [1]:
import pandas as pd
import numpy as np


In [2]:
bloco = pd.read_csv('escala_bloco_dur.csv')
bloco['date'] = pd.to_datetime(bloco['date'], dayfirst=True, format='%d/%m/%Y')
bloco['time'] = pd.to_datetime(bloco['time'], format='%H:%M').dt.time
ordered_weekdays = [
    '1 - Domingo', '2 - Segunda-feira', '3 - Terça-feira',
    '4 - Quarta-feira', '5 - Quinta-feira', '6 - Sexta-feira', '7 - Sábado'
]
bloco['weekday'] = pd.Categorical(bloco['weekday'],
                                  categories=ordered_weekdays,
                                  ordered=True)
bloco = bloco.sort_values(
    ['room', 'weekday', 'turno', 'date', 'time'],
    ignore_index=True
)


In [4]:
# Define keywords for each specialty
keywords = {
    'retina': ['endolaser', 'vitrec', 'retin' 'oleo', 'silic', 'escleral', 'pneumat', 'membranec', 'coroid'],
    'retina clínica': ['quimiot', 'intrav', 'intra-v', 'medicament'], 
    'catarata': ['facec', 'faco', 'lente'], #['faco', 'facectomia', 'lente intra-ocular'],
    'glaucoma': ['antiglau', 'fistuliz', 'trabeculec', 'ciclocrio', 'ciclofoto', 'crioterap', 'agulham', 'iridec'], #['trabecul', 'trabeculectomia', 'ciclo', 'crio', 'glaucoma'],
    'estrabismo': ['estrab'], #['estrabismo'],
    'plástica': ['lacrim', 'evisc', 'enucl', 'lagoft', 'palpeb', 'blefaro', 'ptose', 'entrop', 'ectrop', 'dacrio', 'epila', 'fornix', 'tarsor', 'puntoplastia'], #['blefar', 'palpebra', 'calaz', 'ptose', 'epilacao', 'tarsorrafia', 'cantoplastia'],
    'córnea': ['anel', 'camara', 'cornea', 'cross', 'recobriment', 'ceratec', 'paracent', 'prk'], #['cornea', 'ceratectomia', 'cross link', 'transplante de cornea', 'sutura de cornea']
    'pterígio': ['pter', 'calaz', 'tumor de conjunt', 'tumor conju', 'transplante conju', 'retirada de ponto', 'plastica de conj'], 
    'outro': ['anest']
}
# Specificity order: if multiple matches, choose the first in this list
specificity_order = ['glaucoma', 'retina', 'córnea', 'catarata', 'estrabismo', 'plástica']
def categorize(line):
    found = [spec for spec, keys in keywords.items() if any(key in line.lower() for key in keys)]
    if not found:
        return 'outros'
    for spec in specificity_order:
        if spec in found:
            return spec
    return found[0]

bloco['procedures'] = bloco['procedures'].fillna('').astype(str)
bloco['specialty'] = bloco['procedures'].apply(categorize)


In [6]:
# garante que 'centro' seja string (evita erros com NaN ou int)
bloco['centro'] = bloco['centro'].astype(str)

# cria a coluna 'convenio_type' conforme a primeira letra de 'centro'
bloco['convenio_type'] = np.where(
    bloco['centro'].str.startswith(('7', '8')),
    'Privado',
    'SUS'
)


In [9]:
room = 6
weekday = '6 - Sexta-feira'
turno = '2 - Tarde'

mask = (bloco['room'] == room) & (bloco['weekday'] == weekday) & (bloco['turno'] == turno)

df = bloco[mask]
sala_turno = df.groupby(['date'])

df

,filename,centro,room,weekday,turno,date,time,surgeon,duration_minutes,est_dur,...,anesthesist,patient_name,age,phone,aviso,tipo,cod_paciente,lines,specialty,convenio_type
1842,escala 28 de março - privado.pdf,7 - C. CIRURGICO-LTDA,6,6 - Sexta-feira,2 - Tarde,2025-03-28,14:00:00,EDUARDO BERTARINI MARQUES,NaN,10.0,...,NaN,ROBERTA DO PRA ALANO,26.0,NaN,41615.0,Ambulatorial,309155,['14:00 41615 Ambulatorial 309155 ROBERTA DO P...,córnea,Privado


In [ ]:
mask_specialty = (bloco['specialty'] == 'plástica')
mask_convenio = (bloco['convenio_type'] == 'SUS')

In [252]:
# 1) soma diária por room, weekday, turno
daily = (
    bloco
    .groupby(['weekday','turno','room','date'], as_index=False, observed=False)
    ['est_dur']
    .sum()
    .rename(columns={'est_dur':'daily_sum'})
)

# 1a) remove dias em que daily_sum == 0
daily = daily[daily['daily_sum'] != 0]

daily

,weekday,turno,room,date,daily_sum
993,2 - Segunda-feira,1 - Manhã,1,2025-03-17,150.0
999,2 - Segunda-feira,1 - Manhã,1,2025-03-24,320.0
1005,2 - Segunda-feira,1 - Manhã,1,2025-03-31,120.0
1007,2 - Segunda-feira,1 - Manhã,1,2025-04-14,140.0
1017,2 - Segunda-feira,1 - Manhã,1,2025-04-28,350.0
...,...,...,...,...,...
6450,7 - Sábado,2 - Tarde,5,2025-03-22,240.0
6468,7 - Sábado,2 - Tarde,5,2025-04-26,300.0
6571,7 - Sábado,2 - Tarde,8,2025-08-03,30.0
6720,7 - Sábado,3 - Noite,3,2025-12-04,60.0


In [253]:
# 2) estatísticas (usa só os dias >0)
stats = (
    daily
    .groupby(['weekday','turno', 'room'], observed=True)['daily_sum']
    .agg(min='min', max='max', mean='mean', std='std', )
    .reset_index()
)

mode_df = (
    daily
    .groupby(['room','weekday','turno'], observed=True)['daily_sum']
    .apply(lambda x: x.mode().iat[0] if not x.mode().empty else np.nan)
    .reset_index(name='mode')
)

result = stats.merge(mode_df, on=['room','weekday','turno'])

cols = ['mean','std','max','min','mode']
result[cols] = result[cols].round(0)

result

,weekday,turno,room,min,max,mean,std,mode
0,2 - Segunda-feira,1 - Manhã,1,120.0,350.0,227.0,102.0,120.0
1,2 - Segunda-feira,1 - Manhã,2,120.0,150.0,141.0,15.0,150.0
2,2 - Segunda-feira,1 - Manhã,3,90.0,210.0,124.0,44.0,90.0
3,2 - Segunda-feira,1 - Manhã,4,60.0,210.0,129.0,57.0,60.0
4,2 - Segunda-feira,1 - Manhã,5,110.0,320.0,233.0,79.0,110.0
...,...,...,...,...,...,...,...,...
102,7 - Sábado,2 - Tarde,4,30.0,270.0,130.0,125.0,30.0
103,7 - Sábado,2 - Tarde,5,240.0,300.0,270.0,42.0,240.0
104,7 - Sábado,2 - Tarde,8,30.0,30.0,30.0,NaN,30.0
105,7 - Sábado,3 - Noite,3,60.0,60.0,60.0,NaN,60.0


In [254]:
result['taxa_ocupacao'] = ((result['mean'] / 240) * 100).round(0)
result['periodo'] = result['weekday'].astype(str) + " - " + result['turno'].astype(str)

# 3) pivot: linhas = room, colunas = periodo, valores = taxa_ocupacao
pivot = result.pivot_table(
    index='room',
    columns='periodo',
    values='taxa_ocupacao',
    fill_value=0   # se quiser zeros nos vazios
)

pivot.to_csv('taxa_ocupacao.csv', index=True)


pivot





periodo,2 - Segunda-feira - 1 - Manhã,2 - Segunda-feira - 2 - Tarde,2 - Segunda-feira - 3 - Noite,3 - Terça-feira - 1 - Manhã,3 - Terça-feira - 2 - Tarde,3 - Terça-feira - 3 - Noite,4 - Quarta-feira - 1 - Manhã,4 - Quarta-feira - 2 - Tarde,4 - Quarta-feira - 3 - Noite,5 - Quinta-feira - 1 - Manhã,5 - Quinta-feira - 2 - Tarde,5 - Quinta-feira - 3 - Noite,6 - Sexta-feira - 1 - Manhã,6 - Sexta-feira - 2 - Tarde,6 - Sexta-feira - 3 - Noite,7 - Sábado - 1 - Manhã,7 - Sábado - 2 - Tarde,7 - Sábado - 3 - Noite
room,,,,,,,,,,,,,,,,,,
1,95.0,33.0,41.0,36.0,120.0,42.0,17.0,61.0,35.0,89.0,134.0,69.0,102.0,57.0,43.0,38.0,12.0,0.0
2,59.0,81.0,62.0,36.0,84.0,12.0,37.0,52.0,68.0,48.0,48.0,19.0,80.0,88.0,39.0,52.0,16.0,0.0
3,52.0,57.0,22.0,35.0,123.0,23.0,72.0,48.0,46.0,45.0,108.0,0.0,61.0,40.0,44.0,73.0,40.0,25.0
4,54.0,70.0,62.0,45.0,35.0,30.0,45.0,41.0,32.0,29.0,36.0,17.0,56.0,65.0,43.0,53.0,54.0,0.0
5,97.0,44.0,25.0,75.0,82.0,26.0,104.0,79.0,31.0,62.0,95.0,25.0,93.0,108.0,115.0,91.0,112.0,25.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,6.0,32.0,0.0,4.0,0.0,0.0,0.0,0.0
8,14.0,45.0,26.0,20.0,45.0,45.0,35.0,32.0,15.0,12.0,37.0,8.0,41.0,45.0,8.0,5.0,12.0,0.0
